In [2]:
!mkdir /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats

In [4]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/222/465/GCF_000222465.1_Adig_1.1/GCF_000222465.1_Adig_1.1_rna.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/222/465/GCF_000222465.1_Adig_1.1/GCF_000222465.1_Adig_1.1_protein.faa.gz

--2023-11-01 08:53:20--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/222/465/GCF_000222465.1_Adig_1.1/GCF_000222465.1_Adig_1.1_protein.faa.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.13, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7671951 (7.3M) [application/x-gzip]
Saving to: ‘GCF_000222465.1_Adig_1.1_protein.faa.gz’

     0K .......... .......... .......... .......... ..........  0% 5.72M 1s
    50K .......... .......... .......... .......... ..........  1% 18.0M 1s
   100K .......... .......... .......... .......... ..........  2% 5.68M 1s
   150K .......... .......... .......... .......... ..........  2% 17.1M 1s
   200K .......... .......... .......... .......... ..........  3% 5.96M 1s
   250K .......... .......... .......... .......... ..........  4%  252M 1s
   300K .......... .......... .......... ........

  5100K .......... .......... .......... .......... .......... 68%  341M 0s
  5150K .......... .......... .......... .......... .......... 69%  632M 0s
  5200K .......... .......... .......... .......... .......... 70%  308M 0s
  5250K .......... .......... .......... .......... .......... 70% 7.75M 0s
  5300K .......... .......... .......... .......... .......... 71%  529M 0s
  5350K .......... .......... .......... .......... .......... 72%  644M 0s
  5400K .......... .......... .......... .......... .......... 72%  609M 0s
  5450K .......... .......... .......... .......... .......... 73%  659M 0s
  5500K .......... .......... .......... .......... .......... 74%  147M 0s
  5550K .......... .......... .......... .......... .......... 74%  651M 0s
  5600K .......... .......... .......... .......... .......... 75% 3.04M 0s
  5650K .......... .......... .......... .......... .......... 76%  554M 0s
  5700K .......... .......... .......... .......... .......... 76%  611M 0s
  5750K ....

In [5]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats
gunzip GCF_000222465.1_Adig_1.1_rna.fna.gz
gunzip GCF_000222465.1_Adig_1.1_protein.faa.gz

In [19]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats
cat ./repeatmask.sh
sbatch ./repeatmask.sh

#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=20
#SBATCH --mem=300GB
#SBATCH --time=48:00:00
#SBATCH --job-name=repeatmasker

cd ~
source ~/.bashrc
conda activate repeatmasker
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats

#Make protein blastable database
makeblastdb -in GCF_000222465.1_Adig_1.1_protein.faa -dbtype prot

#blast the proteins to the RepeatMasker Repeat Protein Library
blastp \
	-query GCF_000222465.1_Adig_1.1_protein.faa \
	-db /storage/group/ibb3/default/tools/RepeatMasker/Libraries/RepeatPeps.lib \
	-evalue 1e-5 -culling_limit 2 -num_threads 20 -max_target_seqs 5 \
	-outfmt '6 qseqid sseqid pident evalue bitscore staxids stitle' -out out.br

#remove the predicted TEs found by the blast above from the Adig transcripts 
perl fastaqual_select.pl \
	-f GCF_000222465.1_Adig_1.1_rna.fna \
	-e <(awk '{print $1}' out.br | sort | uniq) > Adigtranscripts.no_tes.fa

#make the no TE Adig transcript file into blast database
makeblastdb -in Adigtranscript

sbatch: slurm_job_submit: No account set, using default acct: open


Submitted batch job 6713642


In [4]:
%%bash
cat /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats/acer_softmasked/acer_draft_no_mito_sorted.fa.tbl

file name: acer_draft_no_mito_sorted.fa
sequences:          6179
total length:  309886885 bp  (309886885 bp excl N/X-runs)
GC level:         38.96 %
bases masked:   58510908 bp ( 18.88 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
SINEs:             5367       851610 bp    0.27 %
      ALUs            0            0 bp    0.00 %
      MIRs            0            0 bp    0.00 %

LINEs:            32981      9655213 bp    3.12 %
      LINE1         417       191286 bp    0.06 %
      LINE2        8221      2305855 bp    0.74 %
      L3/CR1       1866       418346 bp    0.13 %

LTR elements:     11015      4779570 bp    1.54 %
      ERVL            0            0 bp    0.00 %
      ERVL-MaLRs      0            0 bp    0.00 %
      ERV_classI      0            0 bp    0.00 %
      ERV_classII     0            0 bp    0.00 %

DNA elements:     29714      6728276 bp    2.17 %
     hAT

In [22]:
%%bash
source ~/.bashrc
conda activate repeatmasker407
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats/acer_softmasked/
perl /storage/group/ibb3/default/tools/RepeatMasker/util/buildSummary.pl \
    -useAbsoluteGenomeSize acer_draft_no_mito_sorted.fa.out \
    > acer_draft_no_mito_sorted.fa.out.detailed

perl /storage/group/ibb3/default/tools/RepeatMasker/util/calcDivergenceFromAlign.pl \
    -s example \
    acer_draft_no_mito_sorted.fa.cat.gz
    
perl /storage/group/ibb3/default/tools/RepeatMasker/util/createRepeatLandscape.pl \
    -g 309886885 -div example > example.html

..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [43]:
galaxy_meta = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/Galaxy21-[Export_all_sample_data].tabular", 
            sep = "\t")

In [44]:
acer = galaxy_meta[galaxy_meta["Genetic Coral Species Call"] == "A.cervicornis"] #[galaxy_meta["Reef Name"].str.contains("2 parent")]

In [154]:
with open('/storage/group/ibb3/default/acer_linkage_scaffolding/acer_inds.txt', 'w') as f:
    for line in acer["Affymetrix ID"].tolist():
        f.write(f"{line}\n")

In [45]:
included_samples = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/Cross_metadata.csv")

In [ ]:
galaxy_meta = galaxy_meta[galaxy_meta["User Specimen ID"].isin(included_samples["Mote_ind"].tolist())].reset_index(drop=True)

In [15]:
with open('/storage/group/ibb3/default/acer_linkage_scaffolding/mote_cross_inds.txt', 'w') as f:
    for line in galaxy_meta["Affymetrix ID"].tolist():
        f.write(f"{line}\n")

In [25]:
(galaxy_meta[["Affymetrix ID", "User Specimen ID"]]
 .set_index("Affymetrix ID")
 .to_csv("/storage/group/ibb3/default/acer_linkage_scaffolding/reheader.txt", 
         header = None, sep = " "))

In [23]:
!mkdir -p /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding

In [27]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
bcftools view -S mote_cross_inds.txt \
    all_genotyped_samples_including_parents.vcf -O v \
    > mote_samples.vcf

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [28]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
bcftools reheader -s reheader.txt mote_samples.vcf \
    > mote_samples_rehead.vcf

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [51]:
plate2 = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/20220127.MoteMarineLab.Coral30K.Plate2-CoralProbes.GenotypingReport-Best&RecommendedProbes.txt", 
                     sep = "\t", skiprows = 1)

In [53]:
plate1 = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/20220127.MoteMarineLab.Coral30K.Plate1-CoralProbes.GenotypingReport-Best&RecommendedProbes.txt", 
                     sep = "\t", skiprows = 1)

In [54]:
recommended = pd.concat([plate1[["probeset_id/SampleID", "Affy_SNP_ID"]], 
                         plate1[["probeset_id/SampleID", "Affy_SNP_ID"]]]).reset_index(drop=True)

In [55]:
recommended = recommended.sort_values("Affy_SNP_ID").drop_duplicates("Affy_SNP_ID")

In [56]:
recommendedsnps = recommended["probeset_id/SampleID"].tolist()

recommendedsnps = [i for i in plate1["probeset_id/SampleID"].tolist() if i in plate2["probeset_id/SampleID"].tolist()]

In [15]:
!rm /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/acer_recommended_snps.txt

In [16]:
with open('/storage/group/ibb3/default/convert_snpchip_coords/acer_recommended_snps.txt', 'w') as f:
    for line in recommendedsnps:
        f.write(f"{line}\n")

In [59]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding
bwa index /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats/acer_softmasked/acer_draft_no_mito_sorted.fa.masked
bwa mem \
    /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats/acer_softmasked/acer_draft_no_mito_sorted.fa.masked \
    probes.fasta \
    > probes_aln_acer.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


[bwa_index] Pack FASTA... 1.66 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=619773770, availableWord=55609092
[BWTIncConstructFromPacked] 10 iterations done. 88600490 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 166686746 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 236085978 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 297764202 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 352579946 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 401296298 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 444591498 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 483068346 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 517262634 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 547650602 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 574

In [60]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding
sambamba view -t 4 --sam-input \
    -F "mapping_quality >= 3 and not (unmapped or secondary_alignment or supplementary or duplicate)" \
    ./probes_aln_acer.sam > ./unique_mappings_acer_probes.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



sambamba 0.7.1
 by Artem Tarasov and Pjotr Prins (C) 2012-2019
    LDC 1.20.0 / DMD v2.090.1 / LLVM7.0.0 / bootstrap LDC - the LLVM D compiler (0.17.6)



In [61]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding
cut -f1,3,4,5,6,10,12 /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/unique_mappings_acer_probes.sam \
    > /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/unique_mappings_acer_probes.tab

In [62]:
probes = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/unique_mappings_acer_probes.tab", comment = "@", sep = "\t", names = ["probe", "contig", "start", "mapq", "cigar", "seq", "mismatch"])

In [63]:
probes = probes[probes["mapq"] > 20].reset_index(drop=True)
probes["npos"] = np.nan
for i in range(len(probes)):
    probes["npos"][i] = probes["seq"][i].find("N")
probes["new_snp_pos"] = np.nan
probes.loc[~probes["cigar"].str.contains("I|D"), "new_snp_pos"] = probes["npos"]

/tmp/ipykernel_2498548/435527431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["npos"][i] = probes["seq"][i].find("N")


In [64]:
for i in range(len(probes)):
    cigar = re.findall('(\d+)?([A-Za-z])', probes["cigar"][i])
    length = len(cigar)
    if (length < 4) and (length > 2):
        if ( cigar[0][1] == "S" ) and ( cigar[1][1] == "M" ) and ( cigar[2][1] == "S" ):
            print
        elif ( cigar[0][1] == "M" ) and ( cigar[2][1] == "M" ):
            if int(cigar[0][0]) > probes["seq"][i].find("N"):
                probes["new_snp_pos"][i] = probes["seq"][i].find("N")
            elif int(cigar[0][0]) < probes["seq"][i].find("N"):
                if cigar[1][1] == "I":
                    if (int(cigar[0][0]) + int(cigar[1][0])) > probes["npos"][i]:
                        pass
                    else:
                        probes["new_snp_pos"][i] = probes["seq"][i].find("N") + int(cigar[1][0])
                elif cigar[1][1] == "D":
                    if (int(cigar[0][0]) + int(cigar[1][0])) > probes["npos"][i]:
                        pass
                    else:
                        probes["new_snp_pos"][i] = probes["seq"][i].find("N") - int(cigar[1][0])
    if (length > 3):
        if ( cigar[0][1] == "S" ) and ( cigar[1][1] == "M" ):
            if (int(cigar[0][0]) + int(cigar[1][0])) >= probes["seq"][i].find("N"):
                probes["new_snp_pos"][i] = probes["seq"][i].find("N")
    if ( cigar[0][1] == "M" ) and ( int(cigar[0][0]) >= probes["seq"][i].find("N") ):
        probes["new_snp_pos"][i] = probes["seq"][i].find("N")

/tmp/ipykernel_2498548/1691801142.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N")
/tmp/ipykernel_2498548/1691801142.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N")
/tmp/ipykernel_2498548/1691801142.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N") - int(cigar[1][0]

In [65]:
probes["new_snp_pos"] = probes["start"] + probes["new_snp_pos"]

In [66]:
probes = probes[probes["new_snp_pos"].notna()].reset_index(drop=True)
#now excluding probes w/ excessive mismatch due to indels --> probes.loc[probes["new_snp_pos"].isna(), "new_snp_pos"] = probes["start"] + probes["npos"]
probes["new_snp_pos"] = probes["new_snp_pos"].astype(int)

In [67]:
recommended = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/acer_recommended_snps.txt", header = None)

In [69]:
probes = probes[probes["probe"].isin(recommended[0].tolist())].reset_index(drop=True)

In [13]:
!rm /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/probes_in_acer.txt

In [70]:
file = open('/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/probes_in_acer.txt','w')
for item in probes["probe"].tolist():
    file.write(item+"\n")
file.close()

In [71]:
reformat = probes[["contig", "start", "probe"]]
reformat.columns = ["contig", "start", "ID"]

In [72]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
grep -v "^##" ./mote_samples_rehead.vcf | cut -f1-3 \
    > ./original_coords.tab

In [73]:
tmp = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/original_coords.tab", sep = "\t")

In [74]:
tmp = tmp.merge(reformat, how = "left", on = "ID")

In [75]:
tmp["contig"] = tmp["contig"].fillna("NOMATCH")
tmp["start"] = tmp["start"].fillna(0)

In [76]:
reformatted_positions = tmp[["contig", "start", "ID"]]
reformatted_positions.columns = ["#CHROM", "POS", "ID"]

In [77]:
reformatted_positions["POS"] = reformatted_positions["POS"].astype(int)

/tmp/ipykernel_2498548/2815286365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reformatted_positions["POS"] = reformatted_positions["POS"].astype(int)


In [78]:
reformatted_positions.set_index("#CHROM").to_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/remapped_coords.tab", sep = "\t")

In [79]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
grep -e "^##fileformat" -e "^##FORMAT" -e "^##INFO" \
    mote_samples_rehead.vcf \
    > mote_samples_rehead_filtered_remapped.vcf

In [80]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
paste /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/remapped_coords.tab \
    <( grep -v "^##" mote_samples_rehead.vcf | cut -f4- ) \
    >> mote_samples_rehead_filtered_remapped.vcf

In [81]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
grep "^#" mote_samples_rehead_filtered_remapped.vcf \
    > mote_samples_rehead_filtered_remapped_sorted.vcf
grep -v "^#" mote_samples_rehead_filtered_remapped.vcf | grep -v "NOMATCH" \
    | sort -k1,1V -k2,2g \
    >> mote_samples_rehead_filtered_remapped_sorted.vcf

In [82]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
vcftools --vcf mote_samples_rehead_filtered_remapped_sorted.vcf \
    --min-alleles 2 --max-alleles 2 --max-missing 0.8 \
    --recode --out mote_samples_rehead_filtered_remapped_sorted

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf mote_samples_rehead_filtered_remapped_sorted.vcf
	--max-alleles 2
	--min-alleles 2
	--max-missing 0.8
	--out mote_samples_rehead_filtered_remapped_sorted
	--recode

After filtering, kept 163 out of 163 Individuals
Outputting VCF file...
After filtering, kept 22093 out of a possible 22117 Sites
Run Time = 8.00 seconds


Before starting, we'll check to verify that all offspring have the expected parentage:

In [83]:
%%bash
source ~/.bashrc
conda activate lepwrap
cat ../../acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/mote_samples_rehead_filtered_remapped_sorted.recode.vcf \
    | java -cp ../../tools/lep-map3-code/bin IBD vcfFile=- numThreads=8 \
    > ../../acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/ibd_from_vcf.txt

filtering markers...
Number of markers = 22093 of which 5644 pass filtering
Number of individuals = 163
Starting pairwise computations...


In [88]:
%%bash
for i in $( cut -f1 ../../acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/offspring_list.txt ); do
    sort -n -r -k 3,3 ../../acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/ibd_from_vcf.txt | grep "${i}" \
        | grep -e "Acer-058" -e "9395" -e "Acer-056" -e "M-L-13_Mote" \
        -e "Acer-028" -e "M-L-47_Mote" -e "Acer-026" -e "Acer-025" -e "M-L-63_Mote" \
        | head -2
done

Acer-026	AC650	0.435	0.237	0.656	0.107
Acer-025	AC650	0.427	0.287	0.573	0.141
Acer-026	AC651	0.563	0.156	0.562	0.282
Acer-025	AC651	0.483	0.24	0.553	0.207
Acer-026	AC652	0.513	0.171	0.632	0.196
Acer-025	AC652	0.438	0.282	0.559	0.159
Acer-026	AC654	0.438	0.217	0.691	0.092
Acer-025	AC654	0.438	0.275	0.575	0.15
Acer-026	AC655	0.53	0.211	0.518	0.271
Acer-025	AC655	0.399	0.3	0.601	0.098
Acer-026	AC656	0.525	0.154	0.642	0.205
Acer-025	AC656	0.429	0.279	0.584	0.137
Acer-025	AC657	0.473	0.249	0.556	0.195
Acer-026	AC657	0.434	0.248	0.637	0.115
Acer-025	AC658	0.456	0.261	0.566	0.173
Acer-026	AC658	0.452	0.222	0.65	0.127
Acer-026	AC659	0.489	0.152	0.72	0.129
Acer-025	AC659	0.484	0.273	0.486	0.241
Acer-026	AC660	0.505	0.141	0.709	0.15
Acer-025	AC660	0.482	0.274	0.487	0.239
Acer-026	AC662	0.465	0.222	0.625	0.153
Acer-025	AC662	0.44	0.271	0.577	0.151
Acer-026	AC663	0.494	0.168	0.676	0.156
Acer-025	AC663	0.362	0.415	0.445	0.14
Acer-026	AC664	0.461	0.215	0.649	0.136
Acer-025	AC664	0.45	0.28	0.539	0.18

Acer-028	AC831	0.485	0.112	0.808	0.081
Acer-028	AC832	0.524	0.133	0.687	0.181
Acer-058	AC832	0.515	0.109	0.753	0.138
Acer-058	AC833	0.505	0.134	0.722	0.143
Acer-028	AC833	0.488	0.135	0.754	0.111
Acer-058	AC834	0.518	0.112	0.74	0.148
Acer-028	AC834	0.504	0.123	0.745	0.132
Acer-058	AC835	0.543	0.102	0.709	0.189
Acer-028	AC835	0.484	0.163	0.707	0.13
Acer-058	AC836	0.542	0.11	0.696	0.194
Acer-028	AC836	0.514	0.138	0.696	0.166
Acer-058	AC837	0.516	0.128	0.712	0.16
Acer-028	AC837	0.477	0.132	0.782	0.087
Acer-058	AC838	0.522	0.106	0.745	0.149
Acer-028	AC838	0.494	0.132	0.748	0.12
Acer-028	AC839	0.515	0.124	0.722	0.153
Acer-058	AC839	0.511	0.116	0.747	0.137
M-L-47_Mote	AC840	0.541	0.093	0.73	0.176
Acer-028	AC840	0.532	0.086	0.763	0.15
M-L-47_Mote	AC841	0.573	0.093	0.668	0.239
Acer-028	AC841	0.543	0.109	0.697	0.195
M-L-47_Mote	AC842	0.551	0.096	0.707	0.197
Acer-028	AC842	0.53	0.097	0.746	0.157
Acer-028	AC843	0.539	0.116	0.69	0.194
M-L-47_Mote	AC843	0.509	0.126	0.731	0.143
M-L-47_Mote	AC844	0.55

It actually looks like we have 8 problematic samples that do not have the expected parentage. We'll remove these from the pedigree and remove them from the vcf. Problematic samples:
AC679
AC801
AC831
AC849
AC851
AC854
AC855
AC858

In [89]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
vcftools --vcf mote_samples_rehead_filtered_remapped_sorted.vcf \
    --min-alleles 2 --max-alleles 2 --max-missing 0.8 \
    --remove-indv AC679 --remove-indv AC801 --remove-indv AC831 \
    --remove-indv AC849 --remove-indv AC851 --remove-indv AC854 \
    --remove-indv AC855 --remove-indv AC858 \
    --recode --out mote_samples_rehead_filtered_remapped_sorted_ibd

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf mote_samples_rehead_filtered_remapped_sorted.vcf
	--max-alleles 2
	--min-alleles 2
	--max-missing 0.8
	--out mote_samples_rehead_filtered_remapped_sorted_ibd
	--recode
	--remove-indv AC679
	--remove-indv AC801
	--remove-indv AC831
	--remove-indv AC849
	--remove-indv AC851
	--remove-indv AC854
	--remove-indv AC855
	--remove-indv AC858

Excluding individuals in 'exclude' list
After filtering, kept 155 out of 163 Individuals
Outputting VCF file...
After filtering, kept 22093 out of a possible 22117 Sites
Run Time = 8.00 seconds


In [86]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
cat ./minimap_denovo.sh
sbatch ./minimap_denovo.sh

#!/bin/bash
#SBATCH --account=open
#SBATCH --nodes=1
#SBATCH --ntasks=20
#SBATCH --mem=128GB
#SBATCH --time=48:00:00
#SBATCH --job-name=minimap2

source ~/.bashrc
conda activate ipyrad
minimap2 -c -x map-ont -t 20 \
    /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats/acer_softmasked/acer_draft_no_mito_sorted.fa.masked \
    /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/raw/acer_nanopore/concatenated_trimmed.fastq.gz \
    > /storage/home/nsl5160/scratch/lepanchor_lr_denovo.paf
Submitted batch job 6715409


In [6]:
import pandas as pd

In [ ]:
mkdir -p /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2
LepWrap 40 config.yml

In [1]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/3_SeparateChromosomes/
cat ./all.LOD.summary | tr -s " " > tmp.txt

In [4]:
lg = pd.read_csv("/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/3_SeparateChromosomes/tmp.txt", sep = " ")

In [5]:
lg[lg.columns[4:]].head(60)

,LOD.4,LOD.5,LOD.6,LOD.7,LOD.8,LOD.9,LOD.10,LOD.11,LOD.12,LOD.13,LOD.14,LOD.15,LOD.16,LOD.17,LOD.18,LOD.19,LOD.20
0,344,667,934,1244,1752,2269,2892,3325,3676,4040,4301,4611,4887,4999,5154,5256,5305
1,1449,531,518,467,441,174,144,140,136,73,71,67,66,62,62,60,59
2,564,457,431,343,263,135,100,95,76,44,43,41,25,21,21,20,19
3,497,387,369,268,227,120,66,60,50,34,32,27,21,21,20,19,17
4,368,361,334,264,217,118,63,57,44,32,28,26,20,20,15,9,9
5,365,349,333,263,176,113,57,56,44,29,28,26,20,18,14,9,8
6,350,345,328,249,164,104,57,49,34,28,27,26,20,15,10,9,7
7,346,322,296,233,164,102,54,48,34,28,26,20,17,13,10,8,7
8,335,310,265,206,130,86,53,44,34,27,22,20,15,13,9,8,7
9,303,309,247,204,128,85,52,37,33,26,21,20,15,12,9,7,7


In [1]:
%%bash
source ~/.bashrc
conda activate ipyrad
stats.sh /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.fa.gz -Xmx4g

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details
A	C	G	T	N	IUPAC	Other	GC	GC_stdev
0.3051	0.1947	0.1947	0.3054	0.0004	0.0000	0.0000	0.3894	0.0306

Main genome scaffold total:         	3308
Main genome contig total:           	4526
Main genome scaffold sequence total:	305.412 MB
Main genome contig sequence total:  	305.290 MB  	0.040% gap
Main genome scaffold N/L50:         	7/20.051 MB
Main genome contig N/L50:           	115/732.734 KB
Main genome scaffold N/L90:         	14/14.959 MB
Main genome contig N/L90:           	489/114.385 KB
Max scaffold length:                	30.188 MB
Max contig length:                  	3.549 MB
Number of scaffolds > 50 KB:        	106
% main genome in scaffolds > 50 KB: 	96.74%


Minimum 	Number        	Number        	Total         	Total         	Scaffold
Scaffold	of            	of            	Scaffold  

In [2]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/
gunzip ./lepwrap_denovo2/12_Fasta/Anchored.scaffolds.fa.gz

In [4]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/
cat busco.sh | sed "s:/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/asmtype/tmpspp_hifiasm_meta_2/purge_haplotigs:/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/:g" \
    | sed "s/curated.fasta/Anchored.scaffolds.fa/g" \
    > ./tmp.sh
sbatch ./tmp.sh

sbatch: slurm_job_submit: No account set, using default acct: open


Submitted batch job 6799765


Lep-Anchor removed some sequences due to lack of marker/long-read overlap support. There may have also been markers but they may have been at chromosome-ends (which LepWrap trims). We'll add them back in (Lep-Anchor tells you which have been removed). We also need to adjust the AGP to say that the linkage evidence is "map" rather than "na" and adjust column 8b to say "yes". 

In [4]:
%%bash
source ~/.bashrc
conda activate ipyrad
bioawk -c fastx '{ print $name, length($seq) }' \
    < /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.fa \
    | sort -rhk2,2 | head -20

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details
LG4	30187719
LG2	26771374
LG6	25261800
LG3	20967837
LG1	20930785
LG5	20557002
LG8	20050764
LG11	18955976
LG10	18532563
LG12	18312447
LG14	17260901
LG13	15821996
LG7	15288465
LG9	14959302
contig_222	402672
contig_231	388787
contig_243	360883
contig_273	316961
contig_297	296914
contig_312	275537


In [5]:
%%bash
cd /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/
echo -e "LG4\tLG1
LG2\tLG2
LG6\tLG3
LG3\tLG4
LG1\tLG5
LG5\tLG6
LG8\tLG7
LG11\tLG8
LG10\tLG9
LG12\tLG10
LG14\tLG11
LG13\tLG12
LG7\tLG13
LG9\tLG14" > rename.tab
paste <( grep "^>" Anchored.scaffolds.fa | grep -v "LG" | sed "s/>//g" ) \
    <( grep "^>" Anchored.scaffolds.fa | grep -v "LG" | sed "s/>//g" ) \
    >> rename.tab

In [6]:
%%bash
cat /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/11_AGP/lepanchor.contigs.all.agp \
    | tr " " "\t" \
    > /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/lepanchor.adjusted_spacing.contigs.all.agp

In [8]:
%%bash
paste <( awk 'FNR==NR { id[$1]=$2; next } { split($1,a,":"); if (a[1] in id) $1=id[a[1]]; print }' \
    /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/rename.tab \
    <( cut -f1 /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/lepanchor.adjusted_spacing.contigs.all.agp ) ) \
    <( cut -f2- /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/lepanchor.adjusted_spacing.contigs.all.agp ) \
    | sed -e "s:no\tna:yes\tmap:g" > /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/11_AGP/lepanchor.adjusted_spacing_len_renamed.contigs.all.agp

In [10]:
%%bash
source ~/.bashrc
conda activate assembly
ragtag.py agp2fa \
    /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/11_AGP/lepanchor.adjusted_spacing_len_renamed.contigs.all.agp \
    /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_repeats/acer_softmasked/acer_draft_no_mito_sorted.fa.masked \
    > /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed.fa

In [11]:
%%bash
source ~/.bashrc
conda activate ipyrad
sortbyname.sh in=/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed.fa \
    out=/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed_sorted.fa \
    length=t ascending=f overwrite=true

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


java -ea -Xmx54640m -Xms54640m -cp /storage/group/ibb3/default/.conda/envs/ipyrad/opt/bbmap-38.90-0/current/ sort.SortByName in=/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed.fa out=/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed_sorted.fa length=t ascending=f overwrite=true
Executing sort.SortByName [in=/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed.fa, out=/storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed_sorted.fa, length=t, ascending=f, overwrite=true]

Input is being processed as unpaired
Finished reading input.
Sorting.
Created a WriteThread for /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/

In [12]:
%%bash
source ~/.bashrc
conda activate ipyrad
bioawk -c fastx '{ print $name, length($seq) }' \
    < /storage/group/ibb3/default/acer_cnat_dcyl_ssid_assembly/acer_linkage_scaffolding/lepwrap_denovo2/12_Fasta/Anchored.scaffolds.len_renamed_sorted.fa \
    | sort -rhk2,2 | head -20

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details
LG1	30187719
LG2	26771374
LG3	25261800
LG4	20967837
LG5	20930785
LG6	20557002
LG7	20050764
LG8	18955976
LG9	18532563
LG10	18312447
LG11	17260901
LG12	15821996
LG13	15288465
LG14	14959302
contig_222	402672
contig_231	388787
contig_243	360883
contig_273	316961
contig_297	296914
contig_312	275537
